In [1]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import MaxPooling2D
from keras.layers import Conv2D, Flatten
import cv2
import pathlib
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tqdm import tqdm
import pandas as pd
from keras.models import load_model
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
model1=load_model('out/best_identification_model.hdf5')
model2=load_model('out/best_identification_model.hdf5')


In [2]:
class cascade:
    def __init__(self):
        self.model_list = []
    def add(self,model):
        self.model_list.append(model)
    def predict(self,array):
        liste=[]
        for model in self.model_list:
            liste2=[]
            for pred in model.predict(array):
                try:
                    liste2.append(list(pred).index(1))
                except:
                    liste2.append(None)
            liste.append(liste2)
            self.prediction=liste
        return liste
    def none(self):
        try:
            model_name=[]
            model_tag=[]
            number_of_none=[]
            number_of_not_none=[]
            i=1
            for liste in self.prediction:
                model_name.append("Model %s"%i)
                model_tag.append(str(self.model_list[i-1]))
                number_of_none.append(self.prediction[i-1].count(None))
                number_of_not_none.append(len(self.prediction[i-1])-self.prediction[i-1].count(None))
                i+=1
            self.df=pd.DataFrame({"model name":model_name,"model tag":model_tag,"number of None": number_of_none,"number of not None": number_of_not_none})
            return self.df
        except AttributeError:
            raise AttributeError("Call .predict() function before calling .none()")
    def get_value(self,i):
        liste=[]
        for k in range(len(self.prediction)):
            if self.prediction[k][i]!=None:
                liste.append(self.prediction[k][i])
        return liste
    
    def tiebreak(self,based_on="most_common",accuracy=None):
        if accuracy is None and based_on=="accuracy_model":
            raise SyntaxError("Accuracy must be a list of your model's accuracy in order")
        if based_on not in ["most_common","accuracy_model"]:
            raise SyntaxError("%s is not a valid parameter, use 'most_common or 'accuracy_model'"%based_on)

        liste=[]
        for k in range(len(self.prediction[0])):
            if based_on=="most_common":
                try:
                    liste.append(Counter(self.get_value(k)).most_common(1)[0][0])
                except:
                    liste.append(None)
            if based_on=="accuracy_model":
                liste2=[]
                for p in set(self.get_value(k)):
                    liste2.append((p,math.prod([1-accuracy[i] for i, x in enumerate(self.get_value(k)) if x == p and p!=None])))
                #liste.append(self.get_value(k)[liste2.index(min(liste2))])
                try:
                    liste.append(list(set(self.get_value(k)))[liste2.index(min(liste2))])
                except:
                    liste.append(None)
        return liste 

In [3]:
def increase_contrast(img):
    clahe = cv2.createCLAHE(clipLimit=50., tileGridSize=(8,8))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels
    l2 = clahe.apply(l)  # apply CLAHE to the L-channel
    lab = cv2.merge((l2,a,b))  # merge channels
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
    return img 

def Preprocessing(image):
    matrix=[]
    image=cv2.imread(r"%s"%image)
    image=increase_contrast(cv2.cvtColor(image, cv2.COLOR_BGR2HLS))
    for k in range(len(image)-1):
        matrix.append(np.round(np.array([image[k][k],image[k][k+1],image[k+1][k],image[k+1][k+1]]).mean()))
    return matrix

 

In [68]:
cas= cascade()
cas.add(model1)
cas.add(model2)
A=cas.predict(X_train)
cas.none()

186/186 [==============================] - 2s 13ms/step


,model name,model tag,number of None,number of not None
0,Model 1,<keras.engine.sequential.Sequential object at ...,153,5787
1,Model 2,<keras.engine.sequential.Sequential object at ...,153,5787


In [60]:
cas.tiebreak(based_on="accuracy_model",accuracy=[0.98,0.88])==cas.tiebreak(based_on="most_common")

True

In [62]:
cas.info()

,model name,model tag,number of None,number of not None
0,Model 1,<keras.engine.sequential.Sequential object at ...,153,5787
1,Model 2,<keras.engine.sequential.Sequential object at ...,153,5787


In [63]:
A


[[348,
  168,
  473,
  386,
  145,
  261,
  271,
  260,
  120,
  36,
  276,
  378,
  248,
  173,
  335,
  121,
  21,
  38,
  176,
  42,
  355,
  357,
  351,
  19,
  46,
  442,
  45,
  308,
  295,
  232,
  238,
  173,
  461,
  25,
  59,
  341,
  131,
  109,
  330,
  1,
  210,
  155,
  425,
  290,
  97,
  395,
  176,
  251,
  99,
  187,
  239,
  469,
  212,
  262,
  7,
  173,
  None,
  65,
  141,
  153,
  255,
  142,
  330,
  321,
  305,
  490,
  None,
  50,
  411,
  129,
  392,
  402,
  265,
  365,
  132,
  259,
  283,
  33,
  2,
  123,
  223,
  49,
  262,
  462,
  155,
  47,
  184,
  163,
  72,
  383,
  204,
  474,
  67,
  123,
  394,
  189,
  86,
  377,
  6,
  100,
  273,
  128,
  14,
  215,
  499,
  405,
  119,
  24,
  301,
  303,
  13,
  85,
  34,
  329,
  494,
  413,
  396,
  453,
  181,
  484,
  67,
  409,
  412,
  None,
  430,
  248,
  316,
  180,
  256,
  141,
  85,
  328,
  177,
  441,
  225,
  86,
  489,
  447,
  346,
  141,
  242,
  406,
  276,
  418,
  215,
  440,
  354,
  9

In [81]:
(1-0.8)*(1-0.3)*(1-0.5)

0.06999999999999998

In [83]:
(1-0.9)*(1-0.92)

0.007999999999999995

In [85]:
a.index(set(a)[0])

TypeError: 'set' object is not subscriptable

In [124]:
import math
for k in set(a):
    print(k,math.prod([1-b[i] for i, x in enumerate(a) if x == k and k!=None]))

2 0.007999999999999995
100 0.06999999999999998
30 0.010000000000000009


In [120]:
cas.nan()['number of None']/len(X_train)


0    0.025758
1    0.025758
Name: number of None, dtype: float64

In [122]:
cas.nan()

,model name,model tag,number of None,number of not None
0,Model 1,<keras.engine.sequential.Sequential object at ...,153,5787
1,Model 2,<keras.engine.sequential.Sequential object at ...,153,5787


In [33]:
a=[0.5,0.8]


0